# Task 3: Training Loop Implementation

**Distributed Data Parallel (DDP)** PyTorch’s built-in features, such as DataParallel (DP) and DistributedDataParallel (DDP) offer accelerated training capabilities. It transparently performs distributed data parallel training.
As an example that uses a torch.nn.Linear as the local model, it is wrapped with DDP, and then runs one forward pass, one backward pass, and an optimizer step on the DDP model. After that, parameters on the local model will be updated, and all models on different processes should be exactly the same.
Library that can be import is from torch.nn.parallel import DistributedDataParallel as DDP

source: https://pytorch.org/docs/master/notes/ddp.html

import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
from torch.nn.parallel import DistributedDataParallel as DDP


def example(rank, world_size):
    # create default process group
    dist.init_process_group("gloo", rank=rank, world_size=world_size)
    # create local model
    model = nn.Linear(10, 10).to(rank)
    # construct DDP model
    ddp_model = DDP(model, device_ids=[rank])
    # define loss function and optimizer
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.001)

    # forward pass
    outputs = ddp_model(torch.randn(20, 10).to(rank))
    labels = torch.randn(20, 10).to(rank)
    # backward pass
    loss_fn(outputs, labels).backward()
    # update parameters
    optimizer.step()

def main():

    world_size = 2
    mp.spawn(example,
        args=(world_size,),
        nprocs=world_size,
        join=True)

if __name__=="__main__":
    # Environment variables which need to be
    # set when using c10d's default "env"
    # initialization mode.
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "29500"
    main()

**Fully Sharded Data Parallel (FSDP)** - This can be used to accelerate training huge models on larger batch sizes. It is a way of training models with parallel processing, similar to traditional data-parallel methods. However, unlike the usual approach that keeps separate copies of a model's parameters, gradients, and optimizer states for each GPU, FSDP divides and shares these states among the parallel workers. Additionally, it provides the option to move the divided model parameters to CPUs if needed.

Normally, FSDP wraps model layers in a nested manner. This means that only the layers within a specific FSDP instance have to bring all the parameters to a single device during forward or backward computations. Once the computation is done, the gathered parameters are released right away. This freed-up memory is then available for the next layer's computation. This process helps save peak GPU memory, allowing for the possibility of training with a larger model size or a larger batch size.

**Using FSDP in pytorch** - There are two ways to wrap a model with PyTorch FSDP. Auto wrapping serves as a seamless replacement for DDP, while manual wrapping requires only minor adjustments to the model definition code, offering the flexibility to experiment with intricate sharding strategies.

Source:https://pytorch.org/blog/introducing-pytorch-fully-sharded-data-parallel-api/

In [ ]:
#Auto wrapping

  from torch.distributed.fsdp import (
    FullyShardedDataParallel,
    CPUOffload,
  )
  from torch.distributed.fsdp.wrap import (
    default_auto_wrap_policy,
  )
  import torch.nn as nn

  class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(8, 4)
        self.layer2 = nn.Linear(4, 16)
        self.layer3 = nn.Linear(16, 4)

  model = DistributedDataParallel(model())
  fsdp_model = FullyShardedDataParallel(
    model(),
    fsdp_auto_wrap_policy=default_auto_wrap_policy,
    cpu_offload=CPUOffload(offload_params=True),
  )